## Import libraries

In [11]:
import spotipy, csv, os, polars as pl
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv

## Authentication

In [12]:
load_dotenv()

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id = os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret = os.getenv("SPOTIFY_CLIENT_SECRET"),
    redirect_uri = os.getenv("SPOTIFY_REDIRECT_URI"),
    scope=(
        "user-library-read user-library-modify "
        "playlist-read-private playlist-read-collaborative "
        "playlist-modify-public playlist-modify-private "
        "user-read-private user-read-email "
        "user-top-read user-follow-read "
        "user-follow-modify user-read-playback-state "
        "user-modify-playback-state user-read-currently-playing "
        "user-read-recently-played app-remote-control "
        "streaming ugc-image-upload"
    )
))

In [13]:
playlist_id = "https://open.spotify.com/playlist/3STWxskDuGGeqe6K1znyQG?si=14f8966f15db4d2a" #My Playlist ('tech')
playlist_URI = playlist_id.split("/")[-1].split("?")[0]

## Getting tracks from playlist

In [14]:
tracks = []
results = sp.playlist_tracks(playlist_id, limit=100, offset=0)
tracks.extend(results['items'])

while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])

print(f"Total songs found: {len(tracks)}")

#Extract data from each track
data = []
for item in tracks:
    track = item['track']
    if track is None:  #There may be deleted songs
        continue

    info = {
        'track_name': track['name'],
        'artists': ', '.join([artist['name'] for artist in track['artists']]),
        'album': track['album']['name'],
        'release_date': track['album']['release_date'],
        'popularity': track['popularity'],
        'duration_ms': track['duration_ms'],
        'explicit': track['explicit'],
        'id': track['id'],
        'type': track['type'],
        'track_number': track['track_number'],

    }
    data.append(info)

Total songs found: 517


## Save / Update .CSV

In [15]:
with open('../data/data.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print("Data saved in data/data.csv")

Data saved in data/data.csv
